### Input files: Price-Today, orders-log
### Output files: Price-for-Order, orders-log, monthly-sales

In [99]:
import glob
import os
import pandas as pd
from datetime import date, timedelta
from pandas.tseries.offsets import *
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()
engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

format_dict = {'qty':'{:,}',
              'sell_price':'{:,.2f}','buy_price':'{:,.2f}','diff':'{:,.2f}',
              'sell_amt':'{:,.2f}','buy_amt':'{:,.2f}','gross':'{:,.2f}','amount':'{:,.2f}',
              'pct':'{:,.2f}','net':'{:,.2f}'}

data_path = "../data/"
csv_path = os.path.join(os.path.expanduser("~"), "iCloudDrive")
one_path = os.path.join(os.path.expanduser("~"), "OneDrive","Documents","Data")
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
dts_path = os.path.join(os.path.expanduser("~"),"Downloads","Datasets")

# Directory to be monitored
directory = os.path.join(os.path.expanduser("~"),"Downloads")

# Dictionary of file categories and their extensions
categories = {
    'Images': ['jpeg', 'jpg', 'png'],
    'PDFs': ['pdf'],
    'Datasets': ['csv', 'xlsx', 'xls', 'json'],
    'Videos': ['mp4','m4a'],
    'Zips': ['zip'],
    'EXEs': ['exe', 'msi'],
    'Pythons': ['ipynb'],
}
pd.options.display.max_rows = 50
pd.options.display.float_format = '{:.2f}'.format
today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2025, 1, 20), datetime.date(2025, 1, 19))

In [101]:
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
yesterday = yesterday.date()
print(f'yesterday: {yesterday}')

today: 2025-01-20
yesterday: 2025-01-17


In [103]:
# If run after actual work day
#today = today - timedelta(days=2)
today

datetime.date(2025, 1, 20)

### Restart and Run All Cells

## Must rerun whenever there is change in orders

In [107]:
sql = """
SELECT trade, name, qty, price AS target, active, reason, market, xdate 
FROM orders
ORDER BY trade, name"""
orders = pd.read_sql(sql, conlite)
orders.shape

(20, 8)

In [109]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
input_file

'../data/Price-Today.csv'

In [111]:
prices = pd.read_csv(input_file)
prices.shape

(176, 10)

In [113]:
df_merge  = pd.merge(orders, prices, on = 'name', how='inner')
df_merge.shape

(20, 17)

In [115]:
df_merge['spd'] = 0

In [117]:
colo = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'price', 'change', '%change',
        'active', 'xdate']
df_merge[colo].shape

(20, 12)

In [119]:
file_name = "Price-for-Order.csv"
data_path = "../data/"
output_file = data_path + file_name
osd_file = osd_path + '\\' + file_name
osd_path = os.path.join(os.path.expanduser("~"),"OneDrive","Documents","obsidian-git-sync","Data")
data_path, output_file, osd_path, osd_file

('../data/',
 '../data/Price-for-Order.csv',
 'C:\\Users\\PC1\\OneDrive\\Documents\\obsidian-git-sync\\Data',
 'C:\\Users\\PC1\\OneDrive\\Documents\\obsidian-git-sync\\Data\\Price-for-Order.csv')

In [121]:
df_merge[colo].to_csv(output_file, header=True, index=False)
df_merge[colo].to_csv(osd_file, header=True, index=False)

### call ruby ruby\order-log-new.rb

In [124]:
%pwd

'C:\\Users\\PC1\\OneDrive\\A6\\Daily'

In [126]:
to_dir = os.path.join(os.path.expanduser("~"),"OneDrive","A5")
os.chdir(to_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [128]:
!ruby ruby\\order-log-new.rb

trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate
B,3BBIF,2,RD50Pct,SET,10000,5.25,5.15,0.05,0.980392,1,2024-11-13
B,AIMIRT,-3,RND05pct,SET,5000,10.2,10.5,0.0,0.0,1,2024-11-26
B,CPNREIT,-11,RD40Pct,SET,5000,10.8,11.9,-0.3,-2.459016,0,2024-11-26
B,DIF,-1,RD40Pct,SET,5000,8.3,8.35,0.05,0.60241,1,2024-11-14
B,GVREIT,-6,RND20Pct,SET,6000,6.25,6.55,-0.1,-1.503759,1,2024-08-28
B,JMART,5,RND65pct,SET50,3200,11.2,10.7,-0.2,-1.834862,1,2024-09-30
B,KCE,8,RD70Pct,SET100,1000,22.8,22.0,0.3,1.382488,1,2024-08-26
B,MCS,-10,RD50Pct,SET,5000,5.8,6.3,-0.05,-0.787402,0,2024-08-23
B,NER,-12,RD40pct,SET,6000,4.7,4.94,0.1,2.066116,0,2024-08-23
B,SENA,-17,RD60pct,SET,7000,1.92,2.18,-0.02,-0.909091,0,2024-05-10
B,TFFIF,-5,RD20Pct,SET,5000,6.0,6.25,0.0,0.0,1,2024-12-12
B,WHAIR,-9,RD30pct,SET,5000,6.0,6.45,0.0,0.0,1,2024-11-21
S,AWC,96,COS,SET50,9000,4.96,3.04,-0.1,-3.184713,0,2024-05-09
S,BCH,75,COS,SET100,2000,21.7,14.2,-0.3,-2.068966,0,2024-08-29
S,JMT,206,COS,SET50,900,50.0,14.3,

In [130]:
to_dir = os.path.join(os.path.expanduser("~"),"OneDrive","A5","Daily")
os.chdir(to_dir)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'

In [132]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
input_file

'../data/orders-log.csv'

In [134]:
df = pd.read_csv(input_file, sep=',', index_col=None)
df['amount'] = df['qty'] * df['target']
df.query('spd >= -3 and spd <= 3')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,2,RD50Pct,SET,10000,5.25,5.15,0.05,0.98,1,2024-11-13,52500.00
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.00,0.00,1,2024-11-26,51000.00
3,B,DIF,-1,RD40Pct,SET,5000,8.30,8.35,0.05,0.60,1,2024-11-14,41500.00


In [136]:
df.query('active == 2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [138]:
df.query('active == 1')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,2,RD50Pct,SET,10000,5.25,5.15,0.05,0.98,1,2024-11-13,52500.00
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.00,0.00,1,2024-11-26,51000.00
3,B,DIF,-1,RD40Pct,SET,5000,8.30,8.35,0.05,0.60,1,2024-11-14,41500.00
4,B,GVREIT,-6,RND20Pct,SET,6000,6.25,6.55,-0.10,-1.50,1,2024-08-28,37500.00
5,B,JMART,5,RND65pct,SET50,3200,11.20,10.70,-0.20,-1.83,1,2024-09-30,35840.00
6,B,KCE,8,RD70Pct,SET100,1000,22.80,22.00,0.30,1.38,1,2024-08-26,22800.00
10,B,TFFIF,-5,RD20Pct,SET,5000,6.00,6.25,0.00,0.00,1,2024-12-12,30000.00
11,B,WHAIR,-9,RD30pct,SET,5000,6.00,6.45,0.00,0.00,1,2024-11-21,30000.00


In [140]:
df.query('name == "AIMIRT"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
1,B,AIMIRT,-3,RND05pct,SET,5000,10.20,10.50,0.00,0.00,1,2024-11-26,51000.00


In [142]:
df.query('active == 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
2,B,CPNREIT,-11,RD40Pct,SET,5000,10.80,11.90,-0.30,-2.46,0,2024-11-26,54000.00
7,B,MCS,-10,RD50Pct,SET,5000,5.80,6.30,-0.05,-0.79,0,2024-08-23,29000.00
8,B,NER,-12,RD40pct,SET,6000,4.70,4.94,0.10,2.07,0,2024-08-23,28200.00
9,B,SENA,-17,RD60pct,SET,7000,1.92,2.18,-0.02,-0.91,0,2024-05-10,13440.00
12,S,AWC,96,COS,SET50,9000,4.96,3.04,-0.10,-3.18,0,2024-05-09,44640.00
13,S,BCH,75,COS,SET100,2000,21.70,14.20,-0.30,-2.07,0,2024-08-29,43400.00
14,S,JMT,206,COS,SET50,900,50.00,14.30,-0.10,-0.69,0,2024-08-22,45000.00
15,S,PTG,64,COS,SET100,3600,11.40,7.50,-0.15,-1.96,0,2024-11-26,41040.00
16,S,RCL,55,RD60pct,SET100,3000,38.75,25.00,0.00,0.00,0,2024-11-21,116250.00
17,S,SYNEX,-481,RD70pct,SET,2000,0.00,13.10,0.00,0.00,0,2024-08-27,0.00


In [144]:
df.query('spd <= -40 and trade == "B"')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount


In [146]:
df.query('trade == "B" & spd > 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
0,B,3BBIF,2,RD50Pct,SET,10000,5.25,5.15,0.05,0.98,1,2024-11-13,52500.00
5,B,JMART,5,RND65pct,SET50,3200,11.20,10.70,-0.20,-1.83,1,2024-09-30,35840.00
6,B,KCE,8,RD70Pct,SET100,1000,22.80,22.00,0.30,1.38,1,2024-08-26,22800.00


In [148]:
df.query('trade == "S" & spd < 0')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount
17,S,SYNEX,-481,RD70pct,SET,2000,0.00,13.10,0.00,0.00,0,2024-08-27,0.00


In [150]:
colx = ['trade', 'name', 'spd', 'reason', 'market', 'qty', 'target', 'current', 'change', '%change',
        'active', 'xdate']

In [154]:
file_name = "orders-log.csv"
output_file = data_path + file_name
osd_file = osd_path + '\\' + file_name
output_file, osd_file

('../data/orders-log.csv',
 'C:\\Users\\PC1\\OneDrive\\Documents\\obsidian-git-sync\\Data\\orders-log.csv')

In [156]:
df[colx].to_csv(output_file, header=True, index=False)
df[colx].to_csv(osd_file, header=True, index=False)

### Convert column name to make it valid for df.query

In [29]:
df['percent'] = df['%change']

In [30]:
df.query('percent < -2')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent


In [31]:
df.query('percent >= 2.00')

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
2,B,CPNREIT,-15,RD40Pct,SET,5000,10.80,12.30,0.30,2.50,1,2024-11-26,54000.00,2.50


In [32]:
df.loc[df.trade == 'B'].sort_values('spd',ascending=False)

,trade,name,spd,reason,market,qty,target,current,change,%change,active,xdate,amount,percent
6,B,KCE,5,RND70pct,SET100,1000,25.00,24.50,-0.20,-0.81,1,2022-08-26,25000.00,-0.81
8,B,NER,4,RD40pct,SET,6000,4.70,4.62,-0.04,-0.86,1,2024-08-23,28200.00,-0.86
0,B,3BBIF,1,RD50Pct,SET,10000,5.25,5.20,-0.10,-1.89,1,2022-11-13,52500.00,-1.89
1,B,AIMIRT,-4,RND05pct,SET,5000,10.20,10.60,0.00,0.00,1,2024-11-26,51000.00,0.00
4,B,GVREIT,-5,RND20Pct,SET,6000,6.25,6.50,0.00,0.00,1,2024-08-28,37500.00,0.00
3,B,DIF,-6,RD40Pct,SET,5000,8.30,8.60,0.00,0.00,1,2024-11-14,41500.00,0.00
10,B,TFFIF,-7,RD20Pct,SET,5000,6.00,6.35,-0.05,-0.78,1,2023-12-12,30000.00,-0.78
11,B,WHAIR,-10,RD30pct,SET,5000,6.00,6.50,-0.05,-0.76,0,2024-11-21,30000.00,-0.76
2,B,CPNREIT,-15,RD40Pct,SET,5000,10.80,12.30,0.30,2.50,1,2024-11-26,54000.00,2.50
7,B,MCS,-18,RD50Pct,SET,5000,5.80,6.70,0.10,1.52,0,2024-08-23,29000.00,1.52


### Sell recently purchased stocks if profitable

In [34]:
pd.set_option('display.max_rows', 40)
sql = """SELECT name, qty, price AS cost, date 
FROM buys B 
JOIN stocks S 
ON stock_id = S.id 
WHERE status = 'Active'
ORDER BY name, date"""
buys = pd.read_sql(sql, conpf)
buys.shape

(114, 4)

In [35]:
colu = 'name date_x qty cost price difa %change %profit'.split()
buys_price = pd.merge(buys, prices, on='name', how='inner')
buys_price['diff'] = buys_price['price'] - buys_price['cost']
buys_price['difa'] = buys_price['diff'] * buys_price['qty']
buys_price['%profit'] = round(buys_price['difa'] / (buys_price['qty'] * buys_price['cost']) * 100,2)

In [36]:
laggards = buys_price.loc[buys_price.cost > buys_price.price,colu]
laggards.shape

(112, 8)

In [37]:
watch_list = buys_price.loc[buys_price.cost <= buys_price.price,colu]
watch_list.sort_values('name',ascending=False)

,name,date_x,qty,cost,price,difa,%change,%profit
33,GVREIT,2023-09-22,6000,6.10,6.55,2700.00,-1.50,7.38
8,AIMIRT,2024-06-17,2500,10.50,10.50,0.00,0.00,0.00


### Profit per Month

In [39]:
year = 2025
month = 1
sql = '''
SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = %s AND MONTH(sells.date) = %s
ORDER BY sells.date DESC, name'''
sql = sql % (year, month)
sells_df = pd.read_sql(sql, conpf)
print(sql)


SELECT name, YEAR(sells.date) AS sell_year, MONTH(sells.date) AS sell_month, 
       sells.price AS sell_price, buys.price AS buy_price, (sells.price - buys.price) AS diff, 
       qty, (sells.price * qty) AS sell_amt, (buys.price * qty) AS buy_amt, 
       (sells.price - buys.price) * qty AS gross, 
       ROUND((sells.price - buys.price)/buys.price*100,2) AS pct, days
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
WHERE YEAR(sells.date) = 2025 AND MONTH(sells.date) = 1
ORDER BY sells.date DESC, name


In [40]:
sells_df

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days


In [41]:
print(f"Monthly Profit Amount: {sells_df.gross.sum():,.2f}")

Monthly Profit Amount: 0.00


In [158]:
file_name = "monthly-sales.csv"
output_file = data_path + file_name
osd_file = osd_path + '\\' + file_name
output_file, osd_file

('../data/monthly-sales.csv',
 'C:\\Users\\PC1\\OneDrive\\Documents\\obsidian-git-sync\\Data\\monthly-sales.csv')

In [160]:
sells_df.to_csv(output_file, header=True, index=False)
sells_df.to_csv(osd_file, header=True, index=False)

In [162]:
sells_df['profit_per_day'] = (sells_df['gross'] / sells_df['buy_amt']) / sells_df['days'] * 100
sells_df.style.format(format_dict)

,name,sell_year,sell_month,sell_price,buy_price,diff,qty,sell_amt,buy_amt,gross,pct,days,profit_per_day


In [164]:
sells_df.gross.sum()

0

In [166]:
average_profit_percent = sells_df['profit_per_day'].mean()
print(f"Average Profit Percent: {average_profit_percent:.2f}%")

Average Profit Percent: nan%


In [168]:
#dt = date(2023, 11, 8)
dt = today
dt.strftime("%B %d, %Y")

'January 20, 2025'

In [170]:
print("Hello, {}! Today is {:%B %d, %Y}.".format("Alice", dt))

Hello, Alice! Today is January 20, 2025.
